In [1]:
from warnings import simplefilter 
simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import time
import pickle
import os
import matplotlib.pyplot as plt
import pandas as pd
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, accuracy_score

from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Input, Conv1D, Flatten
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import GlobalMaxPooling1D, Dropout, MaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
def clean_text(text):
    """Function is used to preprocess user tweets.
    Removes @username and links, and whitespaces.
    Args:
        tweet (str): Raw text of the tweet.
    Returns:
        result (str): Processed tweet, after removing unnecessary data.
    """

    # removing @username
    result = re.sub(r'@[A-Za-z0-9]+', '', text)
    # removing link
    result = re.sub(r'https?://[A-Za-z0-9./]+', '', result)
    # removing leading and trailing whitespace
    result = result.strip()

    return result


def convert_label(polarity):
    """Simple function to preprocess polarity.
    """

    if polarity == 4:
        return 1
    elif polarity == 0:
        return 0
    else:
        print('[WARNING]')


def load_data():
    """Loads the data.
    Function loads in data, preprocesses the data
    (removes @username and links, and converts sentiment
    into neg: 0 and pos: 1)
    Returns:
        texts (pd.Series): Preprocessed texts.
        sentiment (pd.Series): Preprocessed sentiments.
    """

    train_dir = '..\\data\\text\\train.csv'
    columns = ['Polarity', 'ID', 'Date', 'Query', 'User', 'Texts']

    df = pd.read_csv(train_dir, encoding='latin-1', names=columns, header=None)
    df.drop(['ID', 'Date', 'Query', 'User'], axis=1, inplace=True)

    sentiment_raw = df['Polarity']
    sentiment = sentiment_raw.apply(lambda x: convert_label(x))

    texts_raw = df['Texts']
    texts = texts_raw.apply(lambda x: clean_text(x))

    return texts, sentiment

In [4]:
VOCAB_SIZE = 20000
MAX_SEQ_LEN = 40
EMB_DIM = 100
BASE_DIR = str(int(np.ceil(time.time())))

if not os.path.exists(BASE_DIR):
    os.makedirs(BASE_DIR)
    print(f'[INFO] {BASE_DIR} created')

[INFO] 1590446290 created


### Load Data

In [35]:
print('[INFO] Loading data...')
texts, sentiments = load_data()

train_texts, val_texts, train_sent, val_sent = train_test_split(texts, sentiments, test_size=0.2)

[INFO] Loading data...


In [37]:
tk = Tokenizer(num_words=VOCAB_SIZE)
tk.fit_on_texts(train_texts)

In [38]:
with open(f'{BASE_DIR}\\tokenizer.pickle', 'wb') as f:
    pickle.dump(tk, f)

word_index = tk.word_index
print('[INFO] Number of unique tokens found (in train data):', len(word_index))

[INFO] Number of unique tokens found (in train data): 283625


In [39]:
x_train = tk.texts_to_sequences(train_texts)
x_test = tk.texts_to_sequences(val_texts)

max_length = len(max(x_train, key=len))
if max_length > MAX_SEQ_LEN:
    max_length = MAX_SEQ_LEN

In [41]:
x_train = pad_sequences(x_train, maxlen=max_length)
x_test = pad_sequences(x_test, maxlen=max_length)
y_train = np.array(train_sent).reshape(-1, 1)
y_test = np.array(val_sent).reshape(-1, 1)

print(f'[INFO] Sequence Length: {max_length}')
print(f'[INFO] Shape of x_train: {x_train.shape}')
print(f'[INFO] Shape of y_train: {y_train.shape}')
print(f'[INFO] Shape of x_test: {x_test.shape}')
print(f'[INFO] Shape of y_test: {y_test.shape}')

[INFO] Sequence Length: 40
[INFO] Shape of x_train: (1280000, 40)
[INFO] Shape of y_train: (1280000, 1)
[INFO] Shape of x_test: (320000, 40)
[INFO] Shape of y_test: (320000, 1)


### Using Glove Word Embedding
Download glove embedding from : https://nlp.stanford.edu/projects/glove/

File Name: glove.6B.100d.txt

In [12]:
print('[INFO] Indexing word vectors...')
embeddings_index = {}
embedding_path = f'.\glove.6B.{EMB_DIM}d.txt'

with open(embedding_path, encoding='utf8') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('[INFO] Total number of word vectors in Glove Embedding:', len(embeddings_index))

[INFO] Indexing word vectors...
[INFO] Total number of word vectors in Glove Embedding: 400000


In [16]:
print('[INFO] Preparing embedding matrix...')
num_words = min(VOCAB_SIZE, len(word_index) + 1)
embeddings_matrix = np.zeros((num_words, EMB_DIM))  # initializing zeros matrix

for word, i in word_index.items():
    if i >= VOCAB_SIZE:
        continue

    embedding_vector = embeddings_index.get(word)  # vector for that word
    if embedding_vector is not None:  # if word not found, then 0
        embeddings_matrix[i] = embedding_vector

[INFO] Preparing embedding matrix...


In [17]:
embedding_layer = Embedding(num_words, EMB_DIM,
                            weights=[embeddings_matrix],
                            input_length=max_length,
                            trainable=True, name = 'Embedding')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


### Model Architecture

In [19]:
model = Sequential()

model.add(Input(shape=(max_length,), name = 'Input'))
model.add(embedding_layer)
model.add(Dropout(0.2, name = 'Drop_1'))
model.add(Conv1D(512, 3, activation='relu', padding='same', name = 'Conv_1'))
model.add(MaxPooling1D(2, name = 'Max_1'))
model.add(Dropout(0.3, name = 'Drop_2'))

model.add(Conv1D(256, 3, activation='relu', padding='same', name = 'Conv_2'))
model.add(Dropout(0.3, name = 'Drop_3'))

model.add(Conv1D(15, 3, activation='relu', padding='same', name = 'Conv_3'))
model.add(Flatten(name='Flatten_1'))
model.add(Dense(20, name = 'Dense_2'))

model.add(Dense(1, activation='sigmoid', name = 'Output'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print(model.summary())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding (Embedding)        (None, 40, 100)           2000000   
_________________________________________________________________
Drop_1 (Dropout)             (None, 40, 100)           0         
_________________________________________________________________
Conv_1 (Conv1D)              (None, 40, 512)           154112    
_________________________________________________________________
Max_1 (MaxPooling1D)         (None, 20, 512)           0         
_________________________________________________________________
Drop_2 (Dropout)             (None, 20, 512)           0         
________________________________________

### Training

In [24]:
# checkpoint
t = int(time.time())
batch_size = 512

chk_path = os.path.join(BASE_DIR, 'best_{}_{}'.format('Twitter',t))
checkpoint = ModelCheckpoint(chk_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#tensorboard = TensorBoard(log_dir="logs/{}_{}".format('IMDB_LSTM',t))
callbacks_list = [checkpoint]

history = model.fit(x_train, y_train, 
      batch_size=batch_size,
      epochs=100, 
      verbose=1,
      shuffle=True,
      validation_data=[x_test, y_test],
      callbacks=callbacks_list)

#Saving the model
model.save(os.path.join(BASE_DIR, 'final_{}_{}'.format('Twitter',t)))


Train on 1280000 samples, validate on 320000 samples
Epoch 1/100
1278464/1280000 [============================>.] - ETA: 0s - loss: 0.4434 - acc: 0.7912
Epoch 00001: val_acc improved from -inf to 0.82209, saving model to 1590446290\best_Twitter_1590447138
1280000/1280000 [==============================] - 64s 50us/sample - loss: 0.4434 - acc: 0.7912 - val_loss: 0.3964 - val_acc: 0.8221
Epoch 2/100
1278464/1280000 [============================>.] - ETA: 0s - loss: 0.3971 - acc: 0.8207- ETA: - ETA: 5s - loss: 0.3975 - acc: 0.8 - ETA: 4s - loss: 0.3976 - - ETA: 4s - loss: 0.397 - ETA: 3s - loss: 0. - ETA: 1s - 
Epoch 00002: val_acc improved from 0.82209 to 0.82761, saving model to 1590446290\best_Twitter_1590447138
1280000/1280000 [==============================] - 60s 47us/sample - loss: 0.3970 - acc: 0.8207 - val_loss: 0.3837 - val_acc: 0.8276
Epoch 3/100
1279488/1280000 [============================>.] - ETA: 0s - loss: 0.3821 - acc: 0.8289
Epoch 00003: val_acc improved from 0.82761 to

Epoch 26/100
1278464/1280000 [============================>.] - ETA: 0s - loss: 0.2643 - acc: 0.8873
Epoch 00026: val_acc did not improve from 0.83557
1280000/1280000 [==============================] - 66s 51us/sample - loss: 0.2643 - acc: 0.8873 - val_loss: 0.4335 - val_acc: 0.8288
Epoch 27/100
1278976/1280000 [============================>.] - ETA: 0s - loss: 0.2614 - acc: 0.8887
Epoch 00027: val_acc did not improve from 0.83557
1280000/1280000 [==============================] - 66s 51us/sample - loss: 0.2615 - acc: 0.8887 - val_loss: 0.4154 - val_acc: 0.8272
Epoch 28/100
1278464/1280000 [============================>.] - ETA: 0s - loss: 0.2589 - acc: 0.8897- ETA: 9s - loss - ETA: 8s - los - ETA: - ETA: 4s - loss: 0.2585 - ac - ETA: 0s - loss: 0.2588 - acc
Epoch 00028: val_acc did not improve from 0.83557
1280000/1280000 [==============================] - 66s 52us/sample - loss: 0.2589 - acc: 0.8897 - val_loss: 0.4295 - val_acc: 0.8267
Epoch 29/100
1279488/1280000 [==================

1278976/1280000 [============================>.] - ETA: 0s - loss: 0.2001 - acc: 0.9165- ETA: 3 - ETA: 1s - loss: 0.200 - ETA: 0s - loss: 0.2000 - acc: 0
Epoch 00077: val_acc did not improve from 0.83557
1280000/1280000 [==============================] - 66s 52us/sample - loss: 0.2001 - acc: 0.9165 - val_loss: 0.5244 - val_acc: 0.8180
Epoch 78/100
1278976/1280000 [============================>.] - ETA: 0s - loss: 0.1990 - acc: 0.9170- ETA: 7s - loss - ETA: 3s - loss: 0.1986 - acc: 
Epoch 00078: val_acc did not improve from 0.83557
1280000/1280000 [==============================] - 66s 52us/sample - loss: 0.1990 - acc: 0.9170 - val_loss: 0.5413 - val_acc: 0.8175
Epoch 79/100
1279488/1280000 [============================>.] - ETA: 0s - loss: 0.1988 - acc: 0.9170
Epoch 00079: val_acc did not improve from 0.83557
1280000/1280000 [==============================] - 66s 51us/sample - loss: 0.1988 - acc: 0.9170 - val_loss: 0.5304 - val_acc: 0.8177
Epoch 80/100
1279488/1280000 [================

NameError: name 'model_dir' is not defined

### Evaluation

In [65]:
model = load_model(f'{BASE_DIR}//best_Twitter_1590447138')

In [66]:
# Evaluate

y_pred = np.rint(model.predict(x_test))
y_true = y_test
cf_matrix = confusion_matrix(y_true, y_pred)
print(cf_matrix)
class_wise_f1 = f1_score(y_true, y_pred, average=None)
print('the mean-f1 score: {:.4f}'.format(np.mean(class_wise_f1)))
accuracy = accuracy_score(y_true, y_pred)
print('accuracy is: {:.4f}'.format(accuracy))

np.save(f'{BASE_DIR}//y_test_pred', y_pred)

[[134633  25321]
 [ 27296 132750]]
the mean-f1 score: 0.8356
accuracy is: 0.8356
